In [4]:
import pandas as pd

# Carregamento dos dados
caminho_arquivo = 'animes.csv'
dados_animes = pd.read_csv(caminho_arquivo)

# Renomeando e preparando colunas
dados_animes.rename(columns={
    'rate': 'Avaliação do anime',
    'votes': 'Votos totais do anime',
    'episodes': 'Quantidade de episódios do anime'
}, inplace=True)

# Exclusão de animes sem episódios
dados_animes = dados_animes[dados_animes['Quantidade de episódios do anime'] > 0]

# Contagem total de votos
dados_animes['Contagem total de votos de todos os animes'] = dados_animes['Votos totais do anime'].sum()

# Preparação das colunas de gênero e criação de 'Gênero do anime'
colunas_genero = [col for col in dados_animes.columns if 'genre_' in col]
dados_animes['Gênero do anime'] = dados_animes[colunas_genero].apply(lambda x: ', '.join(x.index[x == 1].str.replace('genre_', '').str.capitalize()), axis=1)

# Calculando estatísticas combinadas e armazenando em colunas
for genero in colunas_genero:
    genero_label = genero.replace('genre_', '').capitalize()
    grupo = dados_animes[dados_animes[genero] == 1]
    if not grupo.empty:
        estatisticas = (
            f'Votos: {grupo["Votos totais do anime"].sum()}, '
            f'Animes: {grupo.shape[0]}, '
            f'Episódios: {grupo["Quantidade de episódios do anime"].sum()}, '
            f'Média Avaliações: {grupo["Avaliação do anime"].mean():.2f}, '
            f'Assimetria: {grupo["Avaliação do anime"].skew():.2f}, '
            f'Curtose: {grupo["Avaliação do anime"].kurt():.2f}, '
            f'Desvio Padrão: {grupo["Avaliação do anime"].std():.2f}'
        )
        dados_animes.loc[dados_animes[genero] == 1, 'Estatísticas do gênero'] = estatisticas
        dados_animes.loc[dados_animes[genero] == 1, 'Gênero Calculado'] = genero_label

# Classificação de avaliação
dados_animes['Avaliação Categoria do anime'] = dados_animes['Avaliação do anime'].apply(lambda valor: (
    "Péssimo" if valor <= 1 else
    "Ruim" if valor <= 2 else
    "Neutro" if valor <= 3 else
    "Bom" if valor <= 4 else
    "Excelente"
))

# Salvando o DataFrame processado para exportação
colunas_exportacao = [
    'anime', 'Gênero do anime', 'anime_url', 'anime_img', 'Quantidade de episódios do anime',
    'Votos totais do anime', 'Avaliação do anime', 'Avaliação Categoria do anime',
    'Contagem total de votos de todos os animes', 'Estatísticas do gênero', 'Gênero Calculado'
]

dados_animes_final = dados_animes[colunas_exportacao]
caminho_arquivo_processado = 'animes_para_power_bi.csv'
dados_animes_final.to_csv(caminho_arquivo_processado, index=False)

print(f"Arquivo CSV processado e pronto para uso no Power BI salvo em: {caminho_arquivo_processado}")


Arquivo CSV processado e pronto para uso no Power BI salvo em: animes_para_power_bi.csv
